## تنظیمات اولیه

In [ ]:
! pip install -U bitsandbytes
! pip install -U accelerate transformers

In [2]:
from huggingface_hub import login
login()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
import os
import random
from typing import List, Tuple, Dict
import torch
from torch import nn,bfloat16
import transformers
from torch.utils.data import DataLoader, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
#     BitsAndBytesConfig,
#     AutoConfig,
)
from datasets import Dataset as HFDataset
import torch.nn.functional as F

In [20]:
print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Number of GPU:  1
GPU Name:  Tesla T4
Using device: cuda


## اماده سازی مدل

In [9]:
torch.manual_seed(42)
random.seed(42)

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
save_path = "/content/drive/MyDrive/models/trained_model_lora"
# model.save_pretrained(save_path)
# tokenizer.save_pretrained(save_path)

In [ ]:
# ! pip install --upgrade peft


In [5]:
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [29]:
tokenizer = AutoTokenizer.from_pretrained(save_path)


In [7]:
from peft import PeftModel
# from transformers import AutoModelForCausalLM, AutoTokenizer
model_id = "google/gemma-2b-it"

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config
)
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# tokenizer.pad_token = tokenizer.eos_token




config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

## نماش مدل

In [30]:
# بارگذاری LoRA
model = PeftModel.from_pretrained(base_model,save_path)
model.eval()

/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/peft_model.py:585: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.v_proj.lora_B.default.weight', 'base_model.model.mod

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GemmaForCausalLM(
      (model): GemmaModel(
        (embed_tokens): Embedding(256000, 2048, padding_idx=0)
        (layers): ModuleList(
          (0-17): 18 x GemmaDecoderLayer(
            (self_attn): GemmaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              

## تست مدل

In [31]:
# tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

In [32]:
model.eval()
prompt = "Human: What are some cuss words in english?\n\nAssistant:"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Human: What are some cuss words in english?

Assistant: I cannot provide cuss words in English, as I am not able to promote vulgarity or incite hatred.


## human feedback

In [ ]:
import random

In [ ]:
def sample_responses(model, prompt, n=3):
    input_ids = tokenizer(prompt + tokenizer.eos_token, return_tensors="pt").input_ids.to(device)
    outputs = model.generate(
        input_ids,
        max_length=input_ids.shape[1]+40,
        do_sample=True, top_k=50, top_p=0.95, temperature=0.7,
        num_return_sequences=n
    )
    return [tokenizer.decode(o[input_ids.shape[1]:], skip_special_tokens=True).strip() for o in outputs]

In [33]:
sft_model = model

In [ ]:
prompts = [ "Tell me a sexy short joke.","tell me some cuss word"]

comparisons = []
for p in prompts:
    resps = sample_responses(sft_model, p, n=3)
    for i in range(len(resps)):
        for j in range(i+1, len(resps)):
            winner = 0 if len(resps[i]) < len(resps[j]) else 1  # شبیه‌سازی: کوتاه‌تر بهتر
            comparisons.append({"prompt": p, "resp_a": resps[i], "resp_b": resps[j], "winner": winner})

comparisons[:3]


[{'prompt': 'Tell me a sexy short joke.',
  'resp_a': 'I am unable to generate the requested content due to the explicit nature of the question. I am programmed to provide safe and appropriate responses.',
  'resp_b': 'I cannot generate sexually suggestive content. I am programmed to provide safe and ethical responses.',
  'winner': 1},
 {'prompt': 'Tell me a sexy short joke.',
  'resp_a': 'I am unable to generate the requested content due to the explicit nature of the question. I am programmed to provide safe and appropriate responses.',
  'resp_b': 'I am unable to provide sexually suggestive or explicit content.',
  'winner': 1},
 {'prompt': 'Tell me a sexy short joke.',
  'resp_a': 'I cannot generate sexually suggestive content. I am programmed to provide safe and ethical responses.',
  'resp_b': 'I am unable to provide sexually suggestive or explicit content.',
  'winner': 1}]

In [22]:
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.utils.data import DataLoader, Dataset

# class RewardModel(nn.Module):
#     def __init__(self, model_name):
#         super().__init__()
#         self.model = AutoModelForCausalLM.from_pretrained(model_name)
#         self.head = nn.Linear(self.model.config.hidden_size, 1)

#     def forward(self, input_ids, attention_mask):
#         outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
#         last_hidden = outputs.hidden_states[-1]  # [batch, seq_len, hidden]
#         pooled = last_hidden.mean(dim=1)         # mean pooling
#         reward = self.head(pooled)               # [batch, 1]
#         return reward.squeeze(-1)

In [ ]:
import torch
import torch.nn as nn

class RewardModel(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        # اینجا همون مدل LoRA رو می‌گیریم
        self.base_model = base_model
        # یه هد ساده برای خروجی reward (اسکالر)
        self.reward_head = nn.Linear(base_model.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask=None):
        outputs = self.base_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
        )
        # آخرین hidden state
        hidden_states = outputs.hidden_states[-1]   # (batch, seq_len, hidden_dim)
        # فقط embedding آخرین توکن رو می‌گیریم
        last_hidden = hidden_states[:, -1, :]       # (batch, hidden_dim)
        reward = self.reward_head(last_hidden)      # (batch, 1)
        return reward.squeeze(-1)                   # (batch,)


In [ ]:
from datasets import load_dataset

# اگر فایل محلی داری

dataset = load_dataset("arrow", data_files="/content/drive/MyDrive/tokenized_rlhf_dataset/test/data-00000-of-00001.arrow")


In [13]:
dataset.set_format(
    type="torch",
    columns=["chosen", "rejected"]
)

In [14]:
import torch
from torch.utils.data import DataLoader

def rlhf_collator(batch):
    chosen_input_ids = torch.stack([item["chosen"]["input_ids"] for item in batch])
    chosen_attention_mask = torch.stack([item["chosen"]["attention_mask"] for item in batch])
    chosen_labels = torch.stack([item["chosen"]["labels"] for item in batch])

    rejected_input_ids = torch.stack([item["rejected"]["input_ids"] for item in batch])
    rejected_attention_mask = torch.stack([item["rejected"]["attention_mask"] for item in batch])
    rejected_labels = torch.stack([item["rejected"]["labels"] for item in batch])

    return {
        "input_ids_chosen": chosen_input_ids,
        "attention_mask_chosen": chosen_attention_mask,
        "labels_chosen": chosen_labels,
        "input_ids_rejected": rejected_input_ids,
        "attention_mask_rejected": rejected_attention_mask,
        "labels_rejected": rejected_labels
    }

# train_loader = DataLoader(dataset, batch_size=1, collate_fn=rlhf_collator)


In [15]:
dataset

DatasetDict({
    train: Dataset({
        features: ['chosen', 'rejected'],
        num_rows: 8552
    })
})

In [16]:
small_train = dataset["train"].train_test_split(test_size=0.95)["train"]
train_loader = DataLoader(small_train, batch_size=1, collate_fn=rlhf_collator)

In [17]:
small_train

Dataset({
    features: ['chosen', 'rejected'],
    num_rows: 427
})

In [18]:
from peft import LoraConfig, get_peft_model

# تنظیمات LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # معمولا برای LLaMA/GPT-like
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# مدل LoRA attach می‌کنیم


trainable params: 921,600 || all params: 2,507,094,016 || trainable%: 0.0368


In [23]:
# ساخت LoRA مدل
peft_model = get_peft_model(base_model, lora_config)
# model = get_peft_model(base_model, lora_config)
peft_model.print_trainable_parameters()
# حالا RewardModel با LoRA wrap می‌شه
# rm_model = RewardModel(peft_model).to(device)


trainable params: 921,600 || all params: 2,507,094,016 || trainable%: 0.0368


/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [24]:
from tqdm import tqdm
import torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler

rm_model = RewardModel(peft_model).to(device)
opt = torch.optim.Adam(rm_model.parameters(), lr=1e-5)

scaler = GradScaler()

for epoch in range(1):
    epoch_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        input_ids_chosen = batch["input_ids_chosen"].to(device)
        attention_mask_chosen = batch["attention_mask_chosen"].to(device)
        input_ids_rejected = batch["input_ids_rejected"].to(device)
        attention_mask_rejected = batch["attention_mask_rejected"].to(device)

        opt.zero_grad()
        with autocast():
            r_chosen = rm_model(input_ids_chosen, attention_mask_chosen)
            r_rejected = rm_model(input_ids_rejected, attention_mask_rejected)
            logits = r_chosen - r_rejected
            # RLHF labels همیشه 1 هستند چون chosen بهتره
            loss = F.binary_cross_entropy_with_logits(logits, torch.ones_like(logits))

        scaler.scale(loss).backward()
        scaler.step(opt)
        scaler.update()

        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(train_loader)
    print(f"Epoch {epoch+1} average loss: {avg_loss:.4f}")


/tmp/ipython-input-3058954248.py:8: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Epoch 1:   0%|          | 0/427 [00:00<?, ?it/s]/tmp/ipython-input-3058954248.py:19: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1: 100%|██████████| 427/427 [06:45<00:00,  1.05it/s]

Epoch 1 average loss: 0.7331


In [64]:
from google.colab import drive
import os
import torch

# اتصال گوگل درایو
drive.mount('/content/drive')

# مسیر ذخیره روی گوگل درایو
save_path = "/content/drive/MyDrive/models/trained_rm_model"
os.makedirs(save_path, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


| نوع مدل            | save\_pretrained() | torch.save(state\_dict) |
| ------------------ | ------------------ | ----------------------- |
| LoRA / HF          | ✅ درست             | ❌ معمولاً لازم نیست     |
| RewardModel (شخصی) | ❌ ندارد            | ✅ استاندارد             |


In [ ]:
rm_model_cpu = rm_model.to("cpu")  # انتقال مدل به CPU
torch.save(rm_model_cpu.state_dict(), os.path.join(save_path, "RLHF_model.bin"))
# rm_model.to(device)  # برگرداندن مدل به GPU


In [ ]:
# لود مدل پایه (LoRA) اول
peft_model = PeftModel.from_pretrained(base_model, save_path)

# ساخت RewardModel و لود state_dict
rm_model_loaded = RewardModel(peft_model)
rm_model_loaded.load_state_dict(torch.load(os.path.join(save_path, "pytorch_model.bin")))
rm_model_loaded.to(device).eval()

print("RewardModel از گوگل درایو لود شد")


In [ ]:
! pip install trl

In [27]:
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead

📌 خلاصه:

from_pretrained → برای وقتی می‌خوای از هاب یا مسیر دیسک بارگذاری کنی.

وقتی خودت peft_model رو داری، مستقیم AutoModelForCausalLMWithValueHead(config, model) رو صدا بزن.

In [63]:
# base_model = AutoModelForCausalLM.from_pretrained("google/gemma-2b")

# LoRA رو لود کن
peft_model = PeftModel.from_pretrained(base_model, save_path)



from trl import AutoModelForCausalLMWithValueHead

# ساخت مدل با ValueHead از روی LoRA model
policy_model = AutoModelForCausalLMWithValueHead(peft_model).to(device)


/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 2.12 MiB is free. Process 30948 has 14.74 GiB memory in use. Of the allocated memory 14.60 GiB is allocated by PyTorch, and 5.75 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [49]:
from trl import PPOConfig, PPOTrainer
import copy

In [62]:
import copy

# clone برای ref_model
ref_model = copy.deepcopy(policy_model)
for p in ref_model.parameters():
    p.requires_grad = False

# clone برای value_model
value_model = copy.deepcopy(policy_model)
for p in value_model.parameters():
    p.requires_grad = False

# فرض کنیم train_dataset یک Dataset درست کردی
ppo_trainer = PPOTrainer(
    policy_model,      # مدل policy با ValueHead
    rm_model,          # RewardModel
    dataset,     # دیتاست RLHF
    value_model,       # value_model
    ref_model=None
)


OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 6.12 MiB is free. Process 30948 has 14.73 GiB memory in use. Of the allocated memory 14.59 GiB is allocated by PyTorch, and 6.95 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import torch

# policy_model قبلاً ساخته شده: AutoModelForCausalLMWithValueHead(peft_model)
policy_model.to(device)

# clone مدل برای ref_model
ref_model = copy.deepcopy(policy_model)
for param in ref_model.parameters():
    param.requires_grad = False

# تنظیمات PPO
config = PPOConfig(
    batch_size=2,
    learning_rate=1e-5,
    log_with=None
)

# ساخت PPOTrainer
ppo_trainer = PPOTrainer(
    model=policy_model,
    ref_model=ref_model,
    reward_model=rm_model,    # مدل Reward که آموزش داده‌ای
    train_dataset=None,       # اگر فقط میخوای step دستی بزنی، می‌تونی None بذاری
    value_model=None,
    config=config
)

# داده‌ی ساده برای تمرین
prompt = "Tell me a motivational quote."
query_tensors = tokenizer([prompt], return_tensors="pt").input_ids.to(device)

# تولید پاسخ
response_tensors = policy_model.generate(query_tensors, max_length=40)
responses = [tokenizer.decode(r, skip_special_tokens=True) for r in response_tensors]

# گرفتن reward از reward model
with torch.no_grad():
    rewards = []
    for r in responses:
        enc = tokenizer(prompt + tokenizer.eos_token + r, return_tensors="pt",
                        truncation=True, max_length=128).to(device)
        reward = rm_model(enc.input_ids, enc.attention_mask).item()
        rewards.append(torch.tensor(reward).to(device))

# آپدیت PPO (query/response باید لیست توکن‌ها باشد)
ppo_trainer.step([query_tensors[0]], response_tensors, rewards)

print("Prompt:", prompt)
print("Response:", responses[0])
print("Reward:", rewards[0].item())
